In [77]:
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd

In [78]:
url = "https://www.goodreads.com/list/show/19.Best_for_Book_Clubs"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [79]:
# Inspect the first book to figure what to put in the dictionary
product_prod = soup.find_all('tr', {'itemtype': 'http://schema.org/Book'})
product_prod[0]

<tr itemscope="" itemtype="http://schema.org/Book">
<td class="number" valign="top">1</td>
<td valign="top" width="5%">
<div class="u-anchorTarget" id="4667024"></div>
<div class="js-tooltipTrigger tooltipTrigger" data-resource-id="4667024" data-resource-type="Book">
<a href="/book/show/4667024-the-help" title="The Help">
<img alt="The Help" class="bookCover" itemprop="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1622355533i/4667024._SY75_.jpg"/>
</a> </div>
</td>
<td valign="top" width="100%">
<a class="bookTitle" href="/book/show/4667024-the-help" itemprop="url">
<span aria-level="4" itemprop="name" role="heading">The Help</span>
</a> <br>
<span class="by">by</span>
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<div class="authorName__container">
<a class="authorName" href="https://www.goodreads.com/author/show/1943477.Kathryn_Stockett" itemprop="url"><span itemprop="name">Kathryn Stockett</span></a> <span class="greyTe

In [80]:
books = []
for book in soup.find_all('tr', {'itemtype': 'http://schema.org/Book'}):
    img_url = book.find('img')['src']
    title = book.find('a', {'class': 'bookTitle'}).text.strip()
    author = book.find('a', {'class': 'authorName'}).text.strip()
    rating = book.find('span', {'class': 'minirating'}).text.strip()
    ranking = book.find('td', {'class': 'number'}).text.strip()
    score = book.find('a', {'href': '#'}).text.split(':')[-1].replace(',', '').strip()
    votes = book.find_all('a', {'href': '#'})[1].text.split(' ')[-0].replace(',', '').strip()
    books.append({'img_url': img_url,
                  'title': title,
                  'author': author,
                  'rating': rating,
                  'ranking': ranking,
                  'score': score,
                  'votes': votes
                 })

In [81]:
# Look at the first book
books[0]

{'img_url': 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1622355533i/4667024._SY75_.jpg',
 'title': 'The Help',
 'author': 'Kathryn Stockett',
 'rating': '4.47 avg rating — 2,588,936 ratings',
 'ranking': '1',
 'score': '219223',
 'votes': '2238'}

In [82]:
#df = pd.DataFrame(books)

In [83]:
#df score = soup.text.split(':')[-1].replace(',', '').strip()

In [84]:
# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['books_db']
collection = db['books_collection']

In [85]:
# Insert data into MongoDB
collection.insert_many(books)